In [9]:
import pandas as pd
import pubchempy as pcp
import numpy as np

In [26]:
df = pd.read_csv('data/drug_response.csv')
df.head()

,experiment_id,cell_id,drug_id,dataset_id,tissue_id,cell_name,drug_name,AAC,dataset_name
0,1,2,21,1,9,1321N1,AEW541,0.101705,CCLE
1,2,12,21,1,28,22RV1,AEW541,0.223977,CCLE
2,3,16,21,1,9,42-MG-BA,AEW541,0.121595,CCLE
3,4,20,21,1,38,5637,AEW541,0.109245,CCLE
4,5,24,21,1,38,639-V,AEW541,0.191285,CCLE


In [27]:
drugs = list(df['drug_name'].unique())
drugs[:10]

['AEW541',
 'Nilotinib',
 '17-AAG',
 'PHA-665752',
 'lapatinib',
 'Nutlin-3',
 'AZD0530',
 'Crizotinib',
 'L-685458',
 'Vandetanib']

In [28]:
smiles = {}
cids = {}
for i, drug in enumerate(drugs):
    compound = pcp.get_compounds(drug, 'name')
    if len(compound)>0:
        smiles[drug] = compound[0].isomeric_smiles
        cids[drug] = compound[0].cid
    else:
        print(f'{drug} not found')
    if i%50 == 0:
        print(i)

0
XMD15-27 not found
NPK76-II-72-1 not found
Genentech Cpd 10 not found
XMD14-99 not found
THZ-2-49 not found
TL-1-85 not found
JNK-9L not found
MPS-1-IN-1 not found
50
BHG712 not found
KIN001-055 not found
TL-2-105 not found
JQ12 not found
SB52334 not found
100
THZ-2-102-1 not found
KIN001-244 not found
QL-XI-92 not found
KIN001-236 not found
VNLG/124 not found
WZ3105 not found
KIN001-260 not found
XMD13-2 not found
QL-XII-47 not found
KIN001-266 not found
KIN001-270 not found
KIN001-102 not found
KIN001-135 not found
150
JW-7-24-1 not found
JW-7-52-1 not found
Lisitinib not found
AUY922 not found
WZ-1-84 not found
rTRAIL not found
200
HG-5-113-01 not found
XMD11-85h not found
Cetuximab not found
QL-XII-61 not found
Cisplatin not found
Nutlin-3a (-) not found
681640 not found
250
QL-VIII-58 not found
HG-5-88-01 not found
Nelfivir not found
Trichostatin.A not found
IKK.16 not found
ERKi II (FR180304) not found
L-779405 not found
Glycyl H1152 not found
TCS2312 dihydrochloride not found


In [16]:
cell_lines = list(df['cell_name'].unique())
len(cell_lines)

1412

In [2]:
mm_df = pd.read_csv('../matchmaker/data/DrugCombinationData.tsv', delimiter='\t')
mm_df.head()

,drug_row,drug_col,cell_line_name,synergy_loewe
0,(+)-BICUCULLINE,TEMOZOLOMIDE,T98G,4.436431
1,(+)-JQ1,(+)-JQ1,RD,1.283298
2,(+)-JQ1,717906-29-1,A-673,-17.246800
3,(+)-JQ1,MK-2206,RD,0.847213
4,(+)-JQ1,RUXOLITINIB,L-1236,-30.821293


In [3]:
mm_cell_lines = list(mm_df['cell_line_name'].unique())
len(mm_cell_lines)

81

In [40]:
mm_drugs = list(mm_df['drug_row'].unique())
mm_drugs.extend(list(mm_df['drug_col'].unique()))
mm_drugs = list(set(mm_drugs))
len(mm_drugs)

3040

In [41]:
mm_smiles = {}
mm_cids = {}
for i, drug in enumerate(mm_drugs):
    compound = pcp.get_compounds(drug, 'name')
    if len(compound)>0:
        mm_smiles[drug] = compound[0].isomeric_smiles
        mm_cids[drug] = compound[0].cid
    else:
        print(f'{drug} not found')
    if i%50 == 0:
        print(i)

717906-29-1


In [43]:
compound = pcp.get_compounds('717906-29-1', 'name')
print(compound)

[Compound(11598628)]


In [44]:
compound[0].isomeric_smiles

'CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C(C=C3)NC(=O)C4)S(=O)(=O)C'

In [45]:
mm_smiles['717906-29-1'] = compound[0].isomeric_smiles
mm_cids['717906-29-1'] = compound[0].cid

In [46]:
sm1 = []
sm2 = []
cell_line = []
syn_score = []
indeces = []
for i, row in mm_df.iterrows():
    sm1.append(mm_smiles[row['drug_row']])
    sm2.append(mm_smiles[row['drug_col']])
    cell_line.append(row['cell_line_name'])
    syn_score.append(row['synergy_loewe'])

In [50]:
mm_thesis = pd.DataFrame()
mm_thesis['drug1'] = sm1
mm_thesis['drug2'] = sm2
mm_thesis['cell_line'] = cell_line
mm_thesis['loewe'] = syn_score
mm_thesis.to_csv('data/DrugCombData.csv', sep=',',index=False,header=False)